# Image viewer

This notebook is for inspecting timelapse microscopy data, with associated sinhgle-cell labels and tracks, showing the infection of human macrophages with Mycobacterium Tuberculosis (Mtb), acquired on an Opera Phenix confocal microscope. 

In [2]:
import os
import json
import re
import glob
import logging

import numpy as np
from natsort import natsorted
from tqdm.auto import tqdm
from skimage import io
import dask
import zarr

import napari
from macrohet import dataio, tile, visualise
import btrack


scale_factor = 2 #5.04
ndim = 2

In [3]:
os.getpid()

867013

### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [4]:
expt_ID = 'ND0005'
location = 'SYNO' # 'NEMO' # 'SYNO'
base_dir = f'/mnt/{location}/macrohet_{location.lower()}/data/{expt_ID}/'
metadata_fn = os.path.join(base_dir, 'acquisition/Images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata_fn = glob.glob(os.path.join(base_dir, 'acquisition/Assaylayout/*.xml'))[0]
assay_layout = dataio.read_harmony_metadata(metadata_fn, assay_layout=True,)
metadata

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!
Reading metadata XML file...
Extracting metadata complete!


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0303K1F1P1R1,Ok,r03c03f01p01-ch1sk1fk1fl1.tiff,3,3,1,1,0,1,1,...,0,0.135257497,0,2024-10-06T18:14:34.337+01:00,640,706,40,1.1,0.2,"[[0.999498,0,0,-8.1],[0,-0.999498,0,-1.0],[0,0..."
1,0303K1F1P1R2,Ok,r03c03f01p01-ch2sk1fk1fl1.tiff,3,3,1,1,0,2,1,...,0,0.135257497,0,2024-10-06T18:14:34.57+01:00,488,522,40,1.1,0.1,"[[0.999498,0,0,-8.1],[0,-0.999498,0,-1.0],[0,0..."
2,0303K1F1P2R1,Ok,r03c03f01p02-ch1sk1fk1fl1.tiff,3,3,1,2,0,1,1,...,2E-06,0.135259494,0,2024-10-06T18:14:34.913+01:00,640,706,40,1.1,0.2,"[[0.999498,0,0,-8.1],[0,-0.999498,0,-1.0],[0,0..."
3,0303K1F1P2R2,Ok,r03c03f01p02-ch2sk1fk1fl1.tiff,3,3,1,2,0,2,1,...,2E-06,0.135259494,0,2024-10-06T18:14:35.133+01:00,488,522,40,1.1,0.1,"[[0.999498,0,0,-8.1],[0,-0.999498,0,-1.0],[0,0..."
4,0303K1F1P3R1,Ok,r03c03f01p03-ch1sk1fk1fl1.tiff,3,3,1,3,0,1,1,...,4E-06,0.135261506,0,2024-10-06T18:14:35.477+01:00,640,706,40,1.1,0.2,"[[0.999498,0,0,-8.1],[0,-0.999498,0,-1.0],[0,0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31093,0404K192F9P1R2,Ok,r04c04f09p01-ch2sk192fk1fl1.tiff,4,4,9,1,191,2,1,...,0,0.135189295,57306.39,2024-10-07T10:10:41.33+01:00,488,522,40,1.1,0.1,"[[0.999498,0,0,-8.1],[0,-0.999498,0,-1.0],[0,0..."
31094,0404K192F9P2R1,Ok,r04c04f09p02-ch1sk192fk1fl1.tiff,4,4,9,2,191,1,1,...,2E-06,0.135191306,57306.39,2024-10-07T10:10:41.677+01:00,640,706,40,1.1,0.2,"[[0.999498,0,0,-8.1],[0,-0.999498,0,-1.0],[0,0..."
31095,0404K192F9P2R2,Ok,r04c04f09p02-ch2sk192fk1fl1.tiff,4,4,9,2,191,2,1,...,2E-06,0.135191306,57306.39,2024-10-07T10:10:41.893+01:00,488,522,40,1.1,0.1,"[[0.999498,0,0,-8.1],[0,-0.999498,0,-1.0],[0,0..."
31096,0404K192F9P3R1,Ok,r04c04f09p03-ch1sk192fk1fl1.tiff,4,4,9,3,191,1,1,...,4E-06,0.135193303,57306.39,2024-10-07T10:10:42.237+01:00,640,706,40,1.1,0.2,"[[0.999498,0,0,-8.1],[0,-0.999498,0,-1.0],[0,0..."


### Now to lazily mosaic the images using Dask prior to saving them out as zarr

In [5]:
zarr_fns_to_do =[]
pattern = r'\((\d+), (\d+)\)\.zarr$'
for acq_ID, data in tqdm(assay_layout.iterrows(), total = len(assay_layout)):
    # break
    zarr_fn = f'/mnt/SYNO/macrohet_syno/data/{expt_ID}/acquisition/zarr/{acq_ID}.zarr'
    if not os.path.exists(zarr_fn):
        zarr_dir = (os.path.dirname(zarr_fn))
        os.makedirs(zarr_dir, exist_ok = True)
        zarr_fns_to_do.append(zarr_fn)

  0%|          | 0/42 [00:00<?, ?it/s]

In [6]:
zarr_fns_to_do

['/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/zarr/(3, 1).zarr',
 '/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/zarr/(3, 2).zarr',
 '/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/zarr/(3, 3).zarr',
 '/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/zarr/(3, 4).zarr',
 '/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/zarr/(3, 5).zarr',
 '/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/zarr/(3, 6).zarr',
 '/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/zarr/(3, 7).zarr',
 '/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/zarr/(3, 8).zarr',
 '/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/zarr/(3, 9).zarr',
 '/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/zarr/(3, 10).zarr',
 '/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/zarr/(3, 11).zarr',
 '/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/zarr/(3, 12).zarr',
 '/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/zarr/(4, 3).zarr',
 '/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/zarr/(4, 4).zarr',
 '/mnt/SYNO/macrohet_syno/data/

In [7]:
len(zarr_fns_to_do)

42

In [8]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger()

for zarr_fn in tqdm(zarr_fns_to_do, total=len(zarr_fns_to_do), desc='Iterating over zarr fns to do'):
    match = re.search(pattern, zarr_fn)
    row, column = acq_ID = (int(match.group(1)), int(match.group(2)))

    image_dir = os.path.join(base_dir, 'acquisition/Images')
    try:
        logger.info(f"Processing acquisition ID: {acq_ID}")
        
        logger.info(f"Compiling mosaic for row {row}, column {column} from {image_dir}")
        dask_images = tile.compile_mosaic(image_dir, metadata, row, column).compute()
        logger.info(f"Mosaic compiled successfully for acquisition ID: {acq_ID}")
        
        data = assay_layout.loc[acq_ID]
        logger.info(f"Retrieved assay layout data for acquisition ID: {acq_ID}")
        
        zarr_group = zarr.open(zarr_fn, mode='w')
        logger.info(f"Opened Zarr file: {zarr_fn}")
        
        acq_metadata = data.to_dict()
        acq_metadata['Acquisition ID'] = acq_ID
        acq_metadata['Experiment ID'] = expt_ID
        acq_metadata['Dimensionality'] = 'TCZYX'
        
        logger.info(f"Rechunking images for acquisition ID: {acq_ID}")
        rechunked_images = dask_images.rechunk((1, 1, 1, 6048, 6048))
        
        logger.info(f"Loading image into Zarr for acquisition ID: {acq_ID}")
        dask.array.to_zarr(rechunked_images, zarr_fn, component='images')
        logger.info(f"Image loaded into Zarr for acquisition ID: {acq_ID}")
        
        zarr_group.attrs['metadata'] = acq_metadata
        logger.info(f"Metadata saved for acquisition ID: {acq_ID}")
        
        logger.info(f"Successfully processed acquisition ID: {acq_ID}")
    except Exception as e:
        logger.error(f"An error occurred: {e}")
        logger.error(f"Acquisition ID: {acq_ID}, Error: {e}")
        continue

Iterating over zarr fns to do:   0%|          | 0/42 [00:00<?, ?it/s]

INFO:root:Processing acquisition ID: (3, 1)
INFO:root:Compiling mosaic for row 3, column 1 from /mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images
ERROR:root:An error occurred: Column not found in metadata.
ERROR:root:Acquisition ID: (3, 1), Error: Column not found in metadata.
INFO:root:Processing acquisition ID: (3, 2)
INFO:root:Compiling mosaic for row 3, column 2 from /mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images
ERROR:root:An error occurred: Column not found in metadata.
ERROR:root:Acquisition ID: (3, 2), Error: Column not found in metadata.
INFO:root:Processing acquisition ID: (3, 3)
INFO:root:Compiling mosaic for row 3, column 3 from /mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images


/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r03c03f01p03-ch1sk186fk1fl1.tiff
/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r03c03f01p02-ch1sk186fk1fl1.tiff
/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r03c03f01p01-ch1sk186fk1fl1.tiff
/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r03c03f01p02-ch2sk186fk1fl1.tiff
/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r03c03f01p01-ch2sk186fk1fl1.tiff
/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r03c03f01p03-ch2sk18fk1fl1.tiff
/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r03c03f01p03-ch2sk186fk1fl1.tiff
/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r03c03f01p02-ch1sk18fk1fl1.tiff
/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r03c03f01p01-ch1sk18fk1fl1.tiff
/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r03c03f01p01-ch1sk177fk1fl1.tiff
/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r03c03f01p02-ch1sk177fk1fl1.tiff
/mnt/SYNO/macrohet_syno/data/ND0005

INFO:root:Mosaic compiled successfully for acquisition ID: (3, 3)
INFO:root:Retrieved assay layout data for acquisition ID: (3, 3)
INFO:root:Opened Zarr file: /mnt/SYNO/macrohet_syno/data/ND0005/acquisition/zarr/(3, 3).zarr
INFO:root:Rechunking images for acquisition ID: (3, 3)
INFO:root:Loading image into Zarr for acquisition ID: (3, 3)
INFO:root:Image loaded into Zarr for acquisition ID: (3, 3)
INFO:root:Metadata saved for acquisition ID: (3, 3)
INFO:root:Successfully processed acquisition ID: (3, 3)
INFO:root:Processing acquisition ID: (3, 4)
INFO:root:Compiling mosaic for row 3, column 4 from /mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images


/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r03c04f01p02-ch1sk36fk1fl1.tiff
/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r03c04f01p02-ch2sk36fk1fl1.tiff
/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r03c04f01p01-ch1sk10fk1fl1.tiff/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r03c04f01p01-ch1sk36fk1fl1.tiff

/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r03c04f01p01-ch2sk36fk1fl1.tiff
/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r03c04f01p02-ch1sk55fk1fl1.tiff
/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r03c04f01p01-ch2sk10fk1fl1.tiff
/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r03c04f01p02-ch1sk10fk1fl1.tiff
/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r03c04f01p03-ch1sk10fk1fl1.tiff
/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r03c04f01p03-ch1sk55fk1fl1.tiff
/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r03c04f01p03-ch1sk36fk1fl1.tiff
/mnt/SYNO/macrohet_syno/data/ND0005/acquisi

INFO:root:Mosaic compiled successfully for acquisition ID: (3, 4)
INFO:root:Retrieved assay layout data for acquisition ID: (3, 4)
INFO:root:Opened Zarr file: /mnt/SYNO/macrohet_syno/data/ND0005/acquisition/zarr/(3, 4).zarr
INFO:root:Rechunking images for acquisition ID: (3, 4)
INFO:root:Loading image into Zarr for acquisition ID: (3, 4)
INFO:root:Image loaded into Zarr for acquisition ID: (3, 4)
INFO:root:Metadata saved for acquisition ID: (3, 4)
INFO:root:Successfully processed acquisition ID: (3, 4)
INFO:root:Processing acquisition ID: (3, 5)
INFO:root:Compiling mosaic for row 3, column 5 from /mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images
ERROR:root:An error occurred: Column not found in metadata.
ERROR:root:Acquisition ID: (3, 5), Error: Column not found in metadata.
INFO:root:Processing acquisition ID: (3, 6)
INFO:root:Compiling mosaic for row 3, column 6 from /mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images
ERROR:root:An error occurred: Column not found in metadata

/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r04c04f01p01-ch2sk166fk1fl1.tiff/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r04c04f01p01-ch1sk166fk1fl1.tiff

/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r04c04f01p02-ch1sk66fk1fl1.tiff
/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r04c04f01p03-ch1sk166fk1fl1.tiff
/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r04c04f01p02-ch2sk166fk1fl1.tiff
/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r04c04f01p01-ch1sk66fk1fl1.tiff
/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r04c04f01p02-ch1sk166fk1fl1.tiff
/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r04c04f01p03-ch2sk166fk1fl1.tiff
/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r04c04f01p01-ch2sk66fk1fl1.tiff
/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r04c04f01p03-ch1sk98fk1fl1.tiff
/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images/r04c04f01p01-ch1sk98fk1fl1.tiff
/mnt/SYNO/macrohet_syno/data/ND0005/a

INFO:root:Mosaic compiled successfully for acquisition ID: (4, 4)
INFO:root:Retrieved assay layout data for acquisition ID: (4, 4)
INFO:root:Opened Zarr file: /mnt/SYNO/macrohet_syno/data/ND0005/acquisition/zarr/(4, 4).zarr
INFO:root:Rechunking images for acquisition ID: (4, 4)
INFO:root:Loading image into Zarr for acquisition ID: (4, 4)
INFO:root:Image loaded into Zarr for acquisition ID: (4, 4)
INFO:root:Metadata saved for acquisition ID: (4, 4)
INFO:root:Successfully processed acquisition ID: (4, 4)
INFO:root:Processing acquisition ID: (4, 5)
INFO:root:Compiling mosaic for row 4, column 5 from /mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images
ERROR:root:An error occurred: Column not found in metadata.
ERROR:root:Acquisition ID: (4, 5), Error: Column not found in metadata.
INFO:root:Processing acquisition ID: (4, 6)
INFO:root:Compiling mosaic for row 4, column 6 from /mnt/SYNO/macrohet_syno/data/ND0005/acquisition/Images
ERROR:root:An error occurred: Column not found in metadata

## Quick check that it worked

In [9]:
assay_layout

Strain Compound Concentration ConcentrationEC  Replicate #
Row Column                                                           
3   1         UNI     CTRL             0             EC0            1
    2         UNI     CTRL             0             EC0            2
    3          WT     CTRL             0             EC0            1
    4          WT     CTRL             0             EC0            2
    5          WT      PZA            60            EC50            1
    6          WT      PZA            60            EC50            2
    7          WT      RIF           0.1            EC50            1
    8          WT      RIF           0.1            EC50            2
    9          WT      INH          0.04            EC50            1
    10         WT      INH          0.04            EC50            2
    11         WT      BDQ          0.02            EC50            1
    12         WT      BDQ          0.02            EC50            2
4   3          WT     CTRL             0             EC0            3
    4          WT     CTRL             0             EC0            4
    5          WT      PZA           400            EC99            1
    6          WT      PZA           400            EC99            2
    7          WT      RIF             2            EC99            1
    8          WT      RIF             2            EC99            2
    9          WT      INH             2            EC99            1
    10         WT      INH             2            EC99            2
    11         WT      BDQ           2.5            EC99            1
    12         WT      BDQ           2.5            EC99            2
5   3         RD1     CTRL             0             EC0            1
    4         RD1     CTRL             0             EC0            2
    5         RD1      PZA            60            EC50            1
    6         RD1      PZA            60            EC50            2
    7         RD1      RIF           0.1            EC50            1
    8         RD1      RIF           0.1            EC50            2
    9         RD1      INH          0.04            EC50            1
    10        RD1      INH          0.04            EC50            2
    11        RD1      BDQ          0.02            EC50            1
    12        RD1      BDQ          0.02            EC50            2
6   3         RD1     CTRL             0             EC0            3
    4         RD1     CTRL             0             EC0            4
    5         RD1      PZA           400            EC99            1
    6         RD1      PZA           400            EC99            2
    7         RD1      RIF             2            EC99            1
    8         RD1      RIF             2            EC99            2
    9         RD1      INH             2            EC99            1
    10        RD1      INH             2            EC99            2
    11        RD1      BDQ           2.5            EC99            1
    12        RD1      BDQ           2.5            EC99            2

In [10]:
acq_ID = (5, 3)
zarr_fn = f'/mnt/SYNO/macrohet_syno/data/{expt_ID}/acquisition/zarr/{acq_ID}.zarr'

In [11]:
zarr_group = zarr.open(zarr_fn)

In [12]:
zarr_group.attrs['metadata']

{'Acquisition ID': [5, 5],
 'Compound': 'PZA',
 'Concentration': '60',
 'ConcentrationEC': 'EC50',
 'Dimensionality': 'TCZYX',
 'Experiment ID': 'ND0003',
 'Replicate #': 1,
 'Strain': 'RD1'}

In [17]:
%%time 
images = zarr_group.images[:]

CPU times: user 38 s, sys: 58.3 s, total: 1min 36s
Wall time: 5min 18s


In [18]:
v = napari.Viewer()

v.add_image(images, channel_axis=1)

[<Image layer 'Image' at 0x7f4c9a571bb0>,
 <Image layer 'Image [1]' at 0x7f4e5cbe7f40>]

In [19]:
print()

In [21]:
images.shape

(154, 2, 3, 6048, 6048)

# Testing PIV case

In [22]:
z0_images = images[:,1,0,...]
z2_images = images[:,1,2,...]

In [23]:
import openpiv

ModuleNotFoundError: No module named 'openpiv'

### testing with compression

In [82]:
# Loop through assay_layout
for (row, column), data in tqdm(assay_layout.iterrows(), total=len(assay_layout)):

    if (row, column) == (3,4):
        continue

    # First set of images without compression
    output_fn = f'/mnt/NEMO/home/users/dayn/macrohet_nemo/{expt_ID}/acquisition/zarr/{row, column}.zarr'
    os.makedirs(os.path.dirname(output_fn), exist_ok=True)    
    # if not os.path.exists(output_fn):

    images = tile.compile_mosaic(image_dir, metadata, row, column).compute()

    store = zarr.DirectoryStore(output_fn)

    # rechunk for saving 
    # images = images.rechunk((150, 2, 3, 2016, 2016)).
    images = images.rechunk((150, 2, 3, 1000, 1000))  # Adjust the chunk sizes as needed

    # Save Dask array to Zarr without compression
    images.to_zarr(store, overwrite=True, group='images', compute=True, codec=zarr.Blosc(cname='lz4', clevel=5))
    
    # images.to_zarr(store, overwrite=True, group='images', compute=True)

    zarr_group = zarr.open(store)
    zarr_group.attrs['Row'] = row
    zarr_group.attrs['Column'] = column
    for key, i in zip(data.keys(), data):
        zarr_group.attrs[key] = i

        # # Second set of images with Blosc-LZ4-Bitshuffle-8 compression
        # output_fn_compressed = f'/mnt/NEMO/home/users/dayn/macrohet_nemo/{expt_ID}/acquisition/zarr/compressed/{row, column}_compressed.zarr'
        # # make dirs
        # os.makedirs(os.path.dirname(output_fn_compressed), exist_ok=True)    
        
        # store_compressed = zarr.DirectoryStore(output_fn_compressed)
    
        # # Specify Blosc-LZ4-Bitshuffle-8 compression options
        # compressor = zarr.Blosc(cname='blosclz', clevel=5, shuffle=zarr.Blosc.BITSHUFFLE)
        
        # # Save Dask array to Zarr with Blosc-LZ4-Bitshuffle-8 compression
        # images.to_zarr(store_compressed, overwrite=True, group='images', compressor=compressor,)# compute=True,)
    
        # zarr_group_compressed = zarr.open(store_compressed)
        # zarr_group_compressed.attrs['Row'] = row
        # zarr_group_compressed.attrs['Column'] = column
        # for key, i in zip(data.keys(), data):
        #     zarr_group_compressed.attrs[key] = i

  0%|          | 0/42 [00:00<?, ?it/s]

/home/dayn/miniconda3/envs/brassica/lib/python3.9/site-packages/zarr/creation.py:295: UserWarning: ignoring keyword argument 'group'
  warn("ignoring keyword argument %r" % k)
/home/dayn/miniconda3/envs/brassica/lib/python3.9/site-packages/zarr/creation.py:295: UserWarning: ignoring keyword argument 'codec'
  warn("ignoring keyword argument %r" % k)
/home/dayn/miniconda3/envs/brassica/lib/python3.9/site-packages/zarr/creation.py:295: UserWarning: ignoring keyword argument 'group'
  warn("ignoring keyword argument %r" % k)
/home/dayn/miniconda3/envs/brassica/lib/python3.9/site-packages/zarr/creation.py:295: UserWarning: ignoring keyword argument 'codec'
  warn("ignoring keyword argument %r" % k)
/home/dayn/miniconda3/envs/brassica/lib/python3.9/site-packages/zarr/creation.py:295: UserWarning: ignoring keyword argument 'group'
  warn("ignoring keyword argument %r" % k)
/home/dayn/miniconda3/envs/brassica/lib/python3.9/site-packages/zarr/creation.py:295: UserWarning: ignoring keyword argu

ValueError: total size of new array must be unchanged